In [1]:
from tqdm import tqdm

In [2]:
data_path = 'data_ori/'
data_path_pro = 'data_pro/'
import os
if not os.path.exists(data_path_pro):
    os.makedirs(data_path_pro)
import numpy as np
import dgl

# Path: data_pro.ipynb
def load_data(data_path, data_path_pro):
    od_matrix = np.load(data_path + 'od_matrix_700.npy')
    od_matrix = od_matrix.astype(np.float32)
    # 读取eoGraph.dgl
    geoGraph = dgl.load_graphs(data_path + 'geoGraph.dgl')[0][0]
    return od_matrix, geoGraph

# Path: data_pro.ipynb
def get_adj(geoGraph):
    adj = geoGraph.adjacency_matrix()
    adj = adj.to_dense()
    adj = adj.numpy()
    return adj

od_matrix, geoGraph = load_data(data_path, data_path_pro)
adj = get_adj(geoGraph)


/usr/local/anaconda3/envs/torch-1.10-py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
trend_len = 7*24
period_len = 24
closeness_len = 3
prediction_len = 12
slot_num = 12

In [10]:
# 将OD矩阵处理成三通道输入
# Path: data_pro.ipynb
def process_od(od_matrix):
    data_closeness = []
    data_trend = []
    data_period = []
    num_hour = od_matrix.shape[0]
    batch_size = num_hour - trend_len - prediction_len + 1
    data_closeness = np.zeros((batch_size, closeness_len, od_matrix.shape[1], od_matrix.shape[2]))
    data_trend = np.zeros((batch_size, slot_num, od_matrix.shape[1], od_matrix.shape[2]))
    data_period = np.zeros((batch_size, slot_num, od_matrix.shape[1], od_matrix.shape[2]))
    data_prediction = np.zeros((batch_size, prediction_len, od_matrix.shape[1], od_matrix.shape[2]))
    for curh in tqdm(range(num_hour)):
        if (curh - trend_len<0) or (curh + prediction_len>num_hour):
            continue
        data_closeness[curh-trend_len] = od_matrix[curh-closeness_len:curh]
        data_trend[curh-trend_len] = od_matrix[curh-trend_len:curh-trend_len+slot_num]
        data_period[curh-trend_len] = od_matrix[curh-period_len:curh-period_len+slot_num]
        data_prediction[curh-trend_len] = od_matrix[curh:curh+prediction_len]
    return data_closeness, data_trend, data_period,data_prediction
data_closeness, data_trend, data_period,data_prediction = process_od(od_matrix)


 49%|████▊     | 341/700 [00:04<00:05, 59.95it/s]  

: 

: 

In [6]:
# 保存为 data_closeness, data_trend, data_period,data_prediction 为npz
np.savez(data_path_pro + 'data.npz', closeness = data_closeness, trend = data_trend, period = data_period, prediction = data_prediction)

(521, 3, 361, 361)

False